## Spatial Overlay

A spatial overlay is the process of superimposing layers of geographic data that cover the same area to study the relationship between them. 

They allow you to compare two GeoDataFrames containing polygon or multi-polygon geometries and create a new Geoframe with the new geometries representing the spatial combination and merged properties.

This allows you to answer question like: What roads are within what blocks or tracts or counties?

There are different types of overlays that can be done, the basic idea for each is shown in the graphic below. 
![overlay operations](http://docs.qgis.org/testing/en/_images/overlay_operations.png)

Now we can load up two GeoDataFrames containing (multi) polygon geometries...

The data for this practice will be NYC borough shape files and the wi-fi hotspot locations:
 * [NYC Borough Boundaries](https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Shapefile) - See it in action [here](https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm)
 
 * [NYC Wi-Fi Hotspot Locations](https://data.cityofnewyork.us/api/geospatial/a9we-mtpn?method=export&format=Shapefile) - See it in action [here](https://data.cityofnewyork.us/Social-Services/NYC-Wi-Fi-Hotspot-Locations/a9we-mtpn)

##### Local File Alternatives for fall-back downloads
 * [nyc_boroughs.zip](https://web.dsa.missouri.edu/static/data/geodata/nyc_boroughs.zip)
 * [nyc_wifi.zip](https://web.dsa.missouri.edu/static/data/geodata/nyc_wifi.zip)


In [ ]:
import os
import pandas as pd
import numpy as np
from shapely.geometry import Point
from random import randint, uniform
import matplotlib.pyplot as plt

import urllib.request
import shutil
from pathlib import Path
from zipfile import ZipFile

import geopandas as gpd

## Loading NYC Boroughs and Wi-FI Data

In [ ]:
# Designate the URL for a file
file_URL = 'https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=Shapefile'

# Designate the local filename
local_file_name = 'nyc_boroughs.zip'

# Designate the local file name with a path to a temp directory.
     # Your Repo comes with this folder.  If not, use terminal and 
     # navigate to course folder > module3 and then: mkdir temp
file_Path = Path('../temp/')  
file_Path /= local_file_name
print(file_Path)

# Download the file from `url` and save it locally under `file_Path`:
with urllib.request.urlopen(file_URL) as response,  file_Path.open(mode='w+b') as out_file:
    shutil.copyfileobj(response, out_file)



In [ ]:
# unzip file
to_unzip = ZipFile('../temp/nyc_boroughs.zip', 'r')
unzipped = '../temp/nyc_boroughs_unzipped'
to_unzip.extractall(unzipped)
to_unzip.close()
for file in os.listdir(unzipped):
    if file.endswith(".shp"):
        shape_file =  unzipped + '/' + file

NYC = gpd.GeoDataFrame.from_file(shape_file)        

In [ ]:
# Designate the URL for a file
file_URL = 'https://data.cityofnewyork.us/api/geospatial/a9we-mtpn?method=export&format=Shapefile'

# Designate the local filename
local_file_name = 'nyc_wifi.zip'

# Designate the local file name with a path to a temp directory.
     # Your Repo comes with this folder.  If not, use terminal and 
     # navigate to course folder > module3 and then: mkdir temp
file_Path = Path('../temp/')  
file_Path /= local_file_name

# Download the file from `url` and save it locally under `file_Path`:
with urllib.request.urlopen(file_URL) as response,  file_Path.open(mode='w+b') as out_file:
    shutil.copyfileobj(response, out_file)

# unzip file
to_unzip = ZipFile('../temp/nyc_wifi.zip', 'r')
unzipped = os.getcwd() + '/nyc_wifi_unzipped'
to_unzip.extractall(unzipped)
to_unzip.close()
for file in os.listdir(unzipped):
    if file.endswith(".shp"):
        shape_file =  unzipped + '/' + file

# Read data into GeoDataFrame and view data
NYC_wifi = gpd.GeoDataFrame.from_file(shape_file)

Now lets plot the data on a map to see what we have.

In [ ]:
NYC_wifi.plot(ax=NYC.plot(), color='red', markersize=1)

### A little data munging for the purpose of this example
As noted earlier, overlays compare two GeoDataFrames containing polygon or multi-polygon geometries. Currently, we do not have this, as the NYC Wi-Fi Hotspot Locations consists of point geometries and not polygons. 
So, for the purposes of this example only, I am going to take the points and create polygons for each location. Imagine if you will that the polygons represent the footprint of each Wi-Fi network.
Additionally, before turning the points into polygons I am going to take a random sample of 250 points as the entire data set is large and takes too long to process for just an example. 

In [ ]:
NYC_wifi_poly = NYC_wifi.sample(250)
NYC_wifi_poly['geometry'] = NYC_wifi.buffer(.002)

In [ ]:
NYC_wifi_poly.plot(ax=NYC.plot(), color='red')

### Overlay Function

Now that we have that out of the way, let's take a closer look at the overlay function within Geopandas.

The geopandas.tools.overlay function takes three arguments:
* dataframe 1
* dataframe 2
* how

Where "how" can be one of:
['intersection',
'union',
'identity',
'symmetric_difference',
'difference']

Keep in mind that overlays operate at the dataframe level, not on individual geometries, and the properties from both are retained. 

So, let's identify the areas (and attributes) where the dataframes intersect using the overlay tool.

In [ ]:
from geopandas.tools import overlay
Interdf = overlay(NYC_wifi_poly, NYC, how="union")
Interdf.plot(ax=NYC.plot(color='red'))

Now let’s compare the original data frame and the new data frame. Notice that in the new data frame we see that the attributes from both original GeoDataFrames are retained.

In [ ]:
NYC.head()

In [ ]:
NYC_wifi_poly.head()

In [ ]:
Interdf.head()

## Intersection 
The resulting geodataframe includes all polygon parts that occur in both input layers and all other parts are excluded. It is roughly analogous to AND in logic and multiplication in arithmetic.

Because both of our original datasets cover all of NYC area, when we ran the intersection you probably noticed that we got all of the wifi hotspots and all of the boroughs. Remember overlays operate at the dataframe level, not on individual geometries, and the properties from both are retained.

Let's break down the borough data into the individual boroughs to get a better visualization of what is going on with the overlays.

In [ ]:
Staten_Island = NYC[NYC['boro_name'] == 'Staten Island']
Manhattan = NYC[NYC['boro_name'] == 'Manhattan']
Brooklyn = NYC[NYC['boro_name'] == 'Brooklyn']
Queens = NYC[NYC['boro_name'] == 'Queens']
Bronx = NYC[NYC['boro_name'] == 'Bronx']

In [ ]:
from geopandas.tools import overlay
Interdf = overlay(NYC_wifi_poly, Manhattan, how="intersection")
Interdf.plot(ax=NYC.plot(),color='red')

Now it is your turn. Below is what each of the other "how" options in the overlay function do. Please explore them, and their differing results, until you feel comfortable with what you can do with each of them.

## Union 
The resulting geodataframe includes all polygon parts that occur in either A, or B, or both, thus is the sum of all the parts of both A and B. Different from "identify" in that individual layers are no longer identifiable. It is roughly analogous to OR in logic and addition in arithmetic.

## Symmetric Difference
The result geodataframe includes polygons that occur in one of the layers but not both. It can be derived as either (A union B) subtract (A intersect B), or (A subtract B) union (B subtract A). It is roughly analogous to XOR in logic.

## Identity 
The resulting geodataframe covers the extent of one of the two layers, with the geometery and attributes merged in the area where they overlap. It can be derived as (A subtract B) union (A intersect B).

## Difference 
The resulting geodataframe includes only those polygon parts that occur in one layer but not in another. It is roughly analogous to AND NOT in logic and subtraction in arithmetic.

## Conclusion
This lab covered:
* Vector overlays using the geopanda overlay function 